In [1]:
import pyspark
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml.regression import GeneralizedLinearRegression
conf = SparkConf().setAppName("MLlib")
sc = SparkContext(conf=conf)
# sc = pyspark.SparkContext("local")
sc.setLogLevel("WARN")
# sc.stop() if need to 

spark = SparkSession(sc)

In [2]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
import pandas as pd
from pyspark.sql import SQLContext
from pyspark import SparkContext

# Logistic Regression

In [3]:
data = spark.read.format("csv").option("header", "true").load(r"C:\Users\yaqi dai\Desktop\UCONN\130-THIRD SEMESTER\5671 Financial Data Mining and Big Data Analytics\project_71\Train_small.csv", inferSchema = True)

In [4]:
colname = data.columns
colname.remove('_c0')
colname.remove('up_down')
feature = VectorAssembler(inputCols = colname,outputCol="features")
data2 = feature.transform(data)
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)
normdata = normalizer.transform(data2)
output = normdata.selectExpr("up_down as label", "normFeatures as features")

In [5]:
train, test = output.randomSplit([0.7, 0.3], seed = 300)
lr = LogisticRegression(featuresCol = "features", labelCol = "label", maxIter=100)
lrModel = lr.fit(train)
predictions = lrModel.transform(test)

In [6]:
evaluator = BinaryClassificationEvaluator()
p = evaluator.evaluate(predictions)
print('the accuracy is :',p)

the accuracy is : 0.7382108163206589


# predict

In [7]:
test_data = spark.read.format("csv").option("header", "true").load(r"C:\Users\yaqi dai\Desktop\UCONN\130-THIRD SEMESTER\5671 Financial Data Mining and Big Data Analytics\project_71\Test_small_features.csv", inferSchema = True)

In [8]:
colname = data.columns
colname.remove('_c0')
colname.remove('up_down')
feature = VectorAssembler(inputCols = colname,outputCol="features1")
test_data2 = feature.transform(test_data)
normalizer = Normalizer(inputCol="features1", outputCol="features", p=1.0)
testnorm = normalizer.transform(test_data2)

In [42]:
predict = lrModel.transform(testnorm)

In [44]:
predict.toPandas().to_csv('prediction.csv')

# Random Forest

In [15]:
rf_data = spark.read.format("csv").option("header", "true").load(r"C:\Users\yaqi dai\Desktop\UCONN\130-THIRD SEMESTER\5671 Financial Data Mining and Big Data Analytics\project_71\Train_small.csv", inferSchema = True)

In [21]:
rf_colname = rf_data.columns
rf_colname.remove('_c0')
rf_colname.remove('up_down')
rf_feature = VectorAssembler(inputCols = rf_colname,outputCol="features1")

In [22]:
rf_data2 = feature.transform(rf_data)
rf_normalizer = Normalizer(inputCol="features1", outputCol="normFeatures", p=1.0)
rf_normdata = rf_normalizer.transform(rf_data2)

In [23]:


rf_output = rf_normdata.selectExpr("up_down as label", "normFeatures as features")
train, test = rf_output.randomSplit([0.8, 0.2], seed = 666)
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=100)
rfModel = rf.fit(train)


In [24]:
predictions = rfModel.transform(test)

In [25]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.327148


In [26]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit


C:\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [27]:
pd.set_option('display.max_columns', None)

df = pd.read_csv(r'C:\Users\yaqi dai\Desktop\UCONN\130-THIRD SEMESTER\5671 Financial Data Mining and Big Data Analytics\project_71\Train_small.csv')
df.head()
# y is the ground truth
y = df['up_down']
y.value_counts()

0    32958
1    16694
Name: up_down, dtype: int64

In [28]:
df.head()
train = df.drop(['Unnamed: 0','up_down'], axis=1)
X = train.as_matrix()
X.shape

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


(49652, 212)

In [29]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [30]:
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=666)


modeling_pipeline = make_pipeline(RandomForestClassifier(n_estimators=10))
cv_scores = cross_val_score(modeling_pipeline, X, y, scoring='accuracy')
print("Cross-val accuracy: %f" %cv_scores.mean())

#rf = 
#rf.fit(X_tr, y_tr)
#y_pr = rf.predict(X_te)
#print(accuracy_score(y_te, y_pr))

Cross-val accuracy: 0.656892


# Multilayer perceptron classifier

In [32]:
# Load training data
nn_data = spark.read.format("csv").option("header", "true").load(r"C:\Users\yaqi dai\Desktop\UCONN\130-THIRD SEMESTER\5671 Financial Data Mining and Big Data Analytics\project_71\Train_small.csv", inferSchema = True)

# Split the data into train and test
nn_colname = nn_data.columns
nn_colname.remove('_c0')
nn_colname.remove('up_down')
nn_feature = VectorAssembler(inputCols = nn_colname,outputCol="features1")
nn_data2 = feature.transform(rf_data)
nn_normalizer = Normalizer(inputCol="features1", outputCol="normFeatures", p=1.0)
nn_normdata = nn_normalizer.transform(nn_data2)
nn_output = nn_normdata.selectExpr("up_down as label", "normFeatures as features")
train, test = nn_output.randomSplit([0.6, 0.4], 1234)



In [33]:
# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [212, 100, 50, 10, 2]

In [34]:
# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

In [35]:
# train the model
model = trainer.fit(train)

In [36]:
# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

Test set accuracy = 0.662308930008045
